In [2]:
# 파이토치로 신경망 모델 구현

# 데이터셋 불러오기
from torchvision.datasets import FashionMNIST

fm_train = FashionMNIST(root='.', train=True, download=True)
fm_test = FashionMNIST(root='.', train=False, download=True)
# root매개변수는 다운로드된 데이터를 저장할 위치를 지정
# train매개변수를 True로하면 훈련 데이터를 다운, False인경우 테스트 데이터를 다운

In [3]:
# Data의 속성확인
type(fm_train.data) #텐서는 파이토치의 기본 데이터 구조

# 데이터 크기확인
print(fm_train.data.shape, fm_test.data.shape)

# 타깃 크기확인
print(fm_train.targets.shape, fm_test.targets.shape)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])
torch.Size([60000]) torch.Size([10000])


In [4]:
# 입력과 타깃데이터 준비
train_input = fm_train.data
train_target = fm_train.targets

train_scaled = train_input/255.0
# 훈련세트와 검증세트로 나누기
from sklearn.model_selection import train_test_split

train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)
print(train_scaled.shape, val_scaled.shape)

torch.Size([48000, 28, 28]) torch.Size([12000, 28, 28])


In [5]:
# 모델
import torch.nn as nn

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784,100),  # 은닉층
    nn.ReLU(),
    nn.Linear(100,10)   # 출력층
)

In [6]:
from torchinfo import summary

summary(model, input_size=(32,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [32, 10]                  --
├─Flatten: 1-1                           [32, 784]                 --
├─Linear: 1-2                            [32, 100]                 78,500
├─ReLU: 1-3                              [32, 100]                 --
├─Linear: 1-4                            [32, 10]                  1,010
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
Total mult-adds (M): 2.54
Input size (MB): 0.10
Forward/backward pass size (MB): 0.03
Params size (MB): 0.32
Estimated Total Size (MB): 0.45

In [7]:
# GPU로 훈련시키기 위해 사용여부 설정
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=100, bias=True)
  (2): ReLU()
  (3): Linear(in_features=100, out_features=10, bias=True)
)

In [9]:
# 옵티마이저 클래스에 대한 객체 생성
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters()) # parameters()메서드는 훈련 가능한 모든 모델 파라미터를 전달
# 모델 마지막층에 활성화 함수를 추가하지 않은 이유 -> Crossentropy 클래스에 소프트맥스 함수가 이미 포함됨, 즉 파이토치 모델마지막에 소프트맥스 함수를 추가할 필요X

In [12]:
epochs = 5
batches = int(len(train_scaled)/32)
for epoch in range(epochs):
    model.train() 
    train_loss = 0 # 에포크 손실 초기화
    for i in range(batches):
        inputs = train_scaled[i*32:(i+1)*32].to(device) # 배치입력과 타깃 준비
        targets = train_target[i*32:(i+1)*32].to(device)
        optimizer.zero_grad() # 옵티마이저 그레디언트 초기화
        outputs = model(inputs) # 모델에 입력 전달
        loss = criterion(outputs, targets) # 모델 출력과 타깃으로 손실 계산 (정방향 계산, 순전파) -> 배치에있는 샘플에 대한 손실 평균 
        loss.backward() # 손실 역전파
        optimizer.step() # 모델 파라미터 업데이트
        train_loss += loss.item() # 에포크 손실 기록
    print(f"에포크:{epoch + 1}, 손실:{train_loss/batches:.4f}") # 에포크 손실 출력

에포크:1, 손실:0.2962
에포크:2, 손실:0.2825
에포크:3, 손실:0.2708
에포크:4, 손실:0.2599
에포크:5, 손실:0.2502


In [ ]:
# 검증세트를 사용해 모델 성능 평가
model.eval() # 모델 평가를 알림
with torch.no_grad(): # 그레디언트 계산을 하지 않는다고 알림
    val_scaled = val_scaled.to(device)
    val_target = val_target.to(device)
    outputs = model(val_scaled) #(12000, 10)
    predicts = torch.argmax(outputs, 1)
    corrects = (predicts == val_target).sum().item()

accuracy = corrects/len(val_target)
print(f"검증 정확도: {accuracy:.4f}")

검증 정확도: 0.8843
